This Notebook is for the Models of our Project.

Models include:
- Linear Regression
- Neural Network
- KNN
- Random Forest

Below you can find the Pre-processing, Training, and Testing for Each model

At the end we will conclude with a comparison between each model and discuss results!

In [ ]:
# imports
import pandas as pd
import numpy as np


In [ ]:
# Data collection
data = pd.read_csv('credit_card_fraud.csv', parse_dates=['trans_date_trans_time',])
